# Zonal Averaging

This section demonstrates how to perform Zonal Averaging using UXarray.


In [ ]:
import uxarray as ux
import numpy as np

uxds = ux.open_dataset(
    "../../test/meshfiles/ugrid/outCSne30/outCSne30.ug",
    "../../test/meshfiles/ugrid/outCSne30/outCSne30_vortex.nc",
)

In [ ]:
# set periodic_elements='exclude' for larger datasets
uxds["psi"].plot(cmap="inferno", periodic_elements="split")

## What is a Zonal Average/Mean?

A zonal average (or zonal mean) is a statistical measure that represents the average of a variable along one or more lines of constant latitude. In other words, it's the mean value calculated around the sphere at constant latitudes. 

UXarray currently implements a conservative Zonal Mean, which weights candidate faces by the length of intersection by a given line of constant latitude. 


```{seealso}
[NCL Zonal Average](https://www.ncl.ucar.edu/Applications/zonal.shtml)
```

In [ ]:
zonal_mean_psi = uxds["psi"].zonal_mean()
zonal_mean_psi

The default latitude range is between -90 and 90 degrees with a step size of 10 degrees. 

In [ ]:
(zonal_mean_psi.plot.line() * zonal_mean_psi.plot.scatter(color="red")).opts(
    title="Zonal Average Plot (Default)", xticks=np.arange(-90, 100, 20), xlim=(-95, 95)
)

The range of latitudes can be modified by using the `lat` parameter. It accepts:

* **Single scalar**: e.g., `lat=45`
* **List/array**: e.g., `lat=[10, 20]` or `lat=np.array([10, 20])`
* **Tuple**: e.g., `(min_lat, max_lat, step)`

In [ ]:
zonal_mean_psi_large = uxds["psi"].zonal_mean(lat=(-90, 90, 1))

In [ ]:
(
    zonal_mean_psi_large.plot.line()
    * zonal_mean_psi_large.plot.scatter(color="red", s=1)
).opts(
    title="Zonal Average Plot (Larger Sample)",
    xticks=np.arange(-90, 100, 20),
    xlim=(-95, 95),
)

## Combined Plots

It is often desired to plot the zonal average along side other plots, such as color or contour plots. 

In [ ]:
(
    uxds["psi"].plot(
        cmap="inferno",
        periodic_elements="split",
        height=250,
        width=500,
        colorbar=False,
        ylim=(-90, 90),
    )
    + zonal_mean_psi.plot.line(
        x="psi_zonal_mean",
        y="latitudes",
        height=250,
        width=150,
        ylabel="",
        ylim=(-90, 90),
        xlim=(0.8, 1.2),
        xticks=[0.8, 0.9, 1.0, 1.1, 1.2],
        yticks=[-90, -45, 0, 45, 90],
        grid=True,
    )
).opts(title="Combined Zonal Average & Raster Plot")